In [2]:
import JDM
import pandas as pd

In [3]:
t_search = JDM.Trie() 

def build_mwe(): 
    df = pd.read_csv("MWE/mwe_id_word.csv", sep=";")

    from alive_progress import alive_bar

    with alive_bar(len(df.index), force_tty=None) as bar:
        for index, row in df.iterrows():
            t_search.addword(row['words'].split())
            
            bar()

In [4]:
build_mwe()

|████████████████████████████████████████| 1738444/1738444 [100%] in 1:28.4 (19666.55/s) 


In [48]:
from alive_progress import alive_bar
from statistics import mean, geometric_mean, harmonic_mean

def remove_useless_words(phrase):
    # Enlever la majuscule au début
    phrase = phrase[0].lower() + phrase[1:]

    # Enlever les articles
    phrase = phrase.replace(" le ", "")
    phrase = phrase.replace(" la ", "")
    phrase = phrase.replace(" les ", "")
    phrase = phrase.replace(" un ", "")
    phrase = phrase.replace(" une ", "")
    phrase = phrase.replace(" des ", "")
    phrase = phrase.replace(" du ", "")
    phrase = phrase.replace(" de ", "")

    phrase = phrase.replace("  ", " ")


    return phrase

def found_compound_terms(phrase): 
    #print(t_search.isCT([phrase[0], phrase[1]])) 

    compound_term = []
    phrase_without_compound_terms = []

    i = 0

    while i < len(phrase):
        #print("for i : ", i)
        compound_term = []

        while i < len(phrase) and t_search.isCT(compound_term + [phrase[i]]): 
            #print("while i : ", i)
            compound_term += [phrase[i]]
            i = i + 1

        new_compound_term = ""

        for term in compound_term : 
            new_compound_term += (term + " ")

        new_compound_term = new_compound_term[:len(new_compound_term)-1]

        phrase_without_compound_terms += [new_compound_term]

    return phrase_without_compound_terms

def relationDistance1(A, B, csv=False): 
    DWH1 = open(JDM.PATH_DWH + JDM.DWH_1, "a")

    relation = A.RS.loc[A.RS['node2'] == B.getID()]

    relation_A_B = {}

    with alive_bar(len(relation.index), force_tty=True) as bar:
        for i, row_i in relation.iterrows() : 
            mot1 = A.getMot()
            type_relation = row_i['type']
            name_relation = A.RT.loc[A.RT['rtid'] == row_i['type']]['trgpname'].to_string(index=False)
            weight_relation = row_i['w']
            mot2 = B.getMot() 

            relation_A_B[mot1 + " " + name_relation + " " + mot2] = weight_relation

            if csv: 
                DWH1.writelines(mot1 + ";" + str(type_relation) + ";" + name_relation + ";" + str(weight_relation) + ";" + str(mot2) + "\n")

            bar()

    DWH1.close() 

    relation_A_B_sorted = {}

    for i in sorted(relation_A_B, key=relation_A_B.get, reverse=True):
        relation_A_B_sorted[i] = relation_A_B[i]

    for r in relation_A_B_sorted: 
        print(str(r) + " : " + str(relation_A_B_sorted[r]))

def relationDistance2(A, B, csv=False):
    termes = []

    #with alive_bar(len(A.RS['node2']), force_tty=True, title="Download ") as bar:
    for eidS in A.RS['node2']: 
        terme = A.E.loc[A.E['eid'] == eidS]['name'].to_string(index=False)

        if JDM.Terme.isValidTerme(terme):
            termes.append(terme)
        #bar() 


    termes = list(dict.fromkeys(termes))
    
    DWH2 = open(JDM.PATH_DWH + JDM.DWH_2, "a")
    relation_A_B = {}

    #with alive_bar(len(termes), force_tty=True, title="Search   ") as bar:
    for terme in termes:     
        AB = JDM.Terme(terme)
        relation_A_AB = A.RS.loc[A.RS['node2'] == AB.getID()]

        for i, row_i in relation_A_AB.iterrows(): 
            relation_AB_B = AB.RS.loc[AB.RS['node2'] == B.getID()]

            for j, row_j in relation_AB_B.iterrows():
                #mot1;type_relation_1;name_relation_1;weight_relation_1;mot2;type_relation_2;name_relation_2;weight_relation_2;mot3;mean
                mot1 = A.getMot()
                type_relation_1 = row_i['type']
                name_relation_1 = A.RT.loc[A.RT['rtid'] == row_i['type']]['trgpname'].to_string(index=False)
                name_relation_1 = name_relation_1 if name_relation_1 !=  "Series([], )" else "Null"
                weight_relation_1 = row_i['w']
                mot2 = AB.getMot() 
                type_relation_2 = row_j['type']
                name_relation_2 = AB.RT.loc[AB.RT['rtid'] == row_j['type']]['trgpname'].to_string(index=False)
                name_relation_2 = name_relation_2 if name_relation_2 !=  "Series([], )" else "Null"
                weight_relation_2 = row_j['w']
                mot3 = B.getMot() 
                mean_of_relations = mean([row_i['w'], row_j['w']])
                mean_geo_of_relations = geometric_mean([row_i['w'] if row_i['w'] > 0 else 1, row_j['w'] if row_j['w'] > 0 else 1])
                mean_har_of_relations = harmonic_mean([row_i['w'] if row_i['w'] > 0 else 0, row_j['w'] if row_j['w'] > 0 else 0])


                relation_A_B[mot1 + " " + name_relation_1 + " " + mot2 + " " + name_relation_2 + " " + mot3] = mean_of_relations

                if csv: 
                    DWH2.writelines(mot1 + ";" + 
                                    str(type_relation_1) + ";" + 
                                    name_relation_1 + ";" + 
                                    str(weight_relation_1)  + ";" + 
                                    mot2 + ";" + 
                                    str(type_relation_2) + ";" + 
                                    name_relation_2 + ";" + 
                                    str(weight_relation_2) + ";" + 
                                    mot3 + ";" + 
                                    str(mean_of_relations) + ";" + 
                                    str(mean_geo_of_relations) + ";" + 
                                    str(mean_har_of_relations) + "\n")
            #bar()

    DWH2.close() 

    relation_A_B_sorted = {}

    for i in sorted(relation_A_B, key=relation_A_B.get, reverse=True):
        relation_A_B_sorted[i] = relation_A_B[i]

    for r in relation_A_B_sorted: 
        print(str(r) + " : " + str(relation_A_B_sorted[r]))

def analyser(phrase): 
    # Transformer la phrase en tableau
    phrase = phrase.split(" ")

    # Trouver les termes composées
    phrase = found_compound_terms(phrase)

    # Chercher les relations entre les termes
    list_of_term = []

    for term in phrase: 
        list_of_term.append(JDM.Terme(term))

    for term in list_of_term: 
        print(term.getMot())

    i = 0

    while i < len(list_of_term) - 1 :
        j = i
        while j < len(list_of_term): 
            relationDistance2(list_of_term[i], list_of_term[j])
            j = j + 1

        i = i + 1

    #relationDistance2(list_of_term[0], list_of_term[2])

    print(phrase)

In [49]:
analyser("Le chat blanc boit du lait de chèvre")

Le chat
blanc
boit
du lait
de chèvre
Le chat idée associée chat locution Le chat : 50
Le chat idée associée chat idée associée Le chat : 25.5
Le chat idée associée chat caractéristique blanc : 191
Le chat idée associée chat couleur blanc : 51
Le chat idée associée chat idée associée blanc : 45.5
Le chat r_wiki janvier couleur blanc : 29
Le chat générique oeuvre couleur blanc : 14
Le chat générique roman couleur blanc : 13.5
Le chat r_wiki cristaux couleur blanc : 5.5
Le chat r_wiki mai couleur blanc : 5.5
Le chat r_wiki science-fiction couleur blanc : 3
Le chat idée associée oeuvre couleur blanc : 2
Le chat idée associée roman couleur blanc : 1.5
blanc idée associée couleur idée associée blanc : 708.5
blanc idée associée noir idée associée blanc : 678.5
blanc idée associée couleur contraire blanc : 571.5
blanc idée associée couleur caractéristique blanc : 518
blanc idée associée couleur couleur blanc : 512.5
blanc caractéristique-1 médicament caractéristique blanc : 509.5
blanc idée as